## Code to compute ZW3 index using Goyal et al., (2022) Journal of Climate

"A new zonal wave 3 index for the Southern Hemisphere" by Goyal, R., Jucker, M., Sen Gupta, A., England, M.

DOI: https://doi.org/10.1175/JCLI-D-21-0927.1

For any suggestions / concerns, 
please contact - Dr. Rishav Goyal
rishav.goyal@unsw.edu.au, 
rishav.goyal@outlook.com

## Requirements

You need to download python EOF package from https://ajdawson.github.io/eofs/latest/

In [ ]:
# Load modules

import numpy as np
import scipy as sc
import xarray as xr
import matplotlib.pyplot as plt
from eofs.standard import Eof
import cartopy.crs as ccrs
import cartopy.crs as ccrs
import cartopy
import matplotlib as mpl
import scipy as sc
import cartopy.feature
import matplotlib.path as mpath
from cartopy.util import add_cyclic_point as cycpt
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker

In [ ]:
# Read data

ds = xr.open_dataset('/files/data/v_era5.nc')

In [ ]:
# Subsample data to only consider meridional winds at 500 hPa and from 40S - 70S and then calculate monthly anomalies

v = ds.v.sel(level=500, time=slice('1979','2020')).sel(latitude=slice(-40,-70)).groupby('time.month') - ds.v.sel(level=500, time=slice('1979','2020')).sel(latitude=slice(-40,-70)).groupby('time.month').mean(dim='time')

In [ ]:
# Carry out the EOFs

lat    = v.latitude
coslat = np.cos(np.deg2rad(lat.values)).clip(0.,1.)
wgts   = np.sqrt(coslat)[...,np.newaxis]
solver = Eof(v.values, weights=wgts)
eof    = solver.eofsAsCovariance(neofs=6)
pcs    = solver.pcs(npcs=6, pcscaling=1)
var    = solver.varianceFraction()

In [ ]:
# Calculate the ZW3 magnitude and phase indices

zw3magnitude_index = (pcs[:,0]**2 + pcs[:,1]**2)**0.5

zw3phase_index = np.zeros(len(v[:,0,0]) * np.nan

for i in range(len(v[:,0,0]):
    if((pcs[i,0]>0) and (pcs[i,1]>0)):
        zw3phase_index[i] = np.arctan(pcs[i,1] / pcs[i,0]) * 180 / (np.pi)
    elif((pcs[i,0]<0) and (pcs[i,1]>0)):
        zw3phase_index[i] = (np.arctan(pcs[i,1] / pcs[i,0]) * 180 / (np.pi)) + 180
    elif((pcs[i,0]>0) and (pcs[i,1]<0)):
        zw3phase_index[i] = (np.arctan(pcs[i,1] / pcs[i,0]) * 180 / (np.pi))
    elif((pcs[i,0]<0) and (pcs[i,1]<0)):
        zw3phase_index[i] = (np.arctan(pcs[i,1] / pcs[i,0]) * 180 / (np.pi)) - 180

In [ ]:
# Save all variables in an xarray (can be then exported to netcdf files directly by using .to_netcdf function)

magnitude_index = xr.DataArray(zw3magnitude_index, coords=[v.time], name='zw3index_magnitude')
phase_index     = xr.DataArray(zw3phase_index, coords=[v.time], name='zw3index_phase')
pc1             = xr.DataArray(pcs[:,0], coords=[v.time], name='pc1')
pc2             = xr.DataArray(pcs[:,1], coords=[v.time], name='pc2')